In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from datetime import datetime
import os.path
import sys
import time
import glob
import numpy as np
from six.moves import xrange
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt

from config import *
from imdb import  kitti
from utils.util import *
from nets import *

In [2]:
import os
path = os.getcwd()

In [3]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string(
        'checkpoint', path[:-4]+'/data/best/model.ckpt-49999',
        """Path to the model parameter file.""")
tf.app.flags.DEFINE_string(
        'input_path', path[:-4]+'/data/samples/*',
        """Input lidar scan to be detected. Can process glob input such as """
        """./data/samples/*.npy or single input.""")
tf.app.flags.DEFINE_string(
        'out_dir', path[:-4]+'/data/samples_out/', """Directory to dump output.""")
tf.app.flags.DEFINE_string('gpu', '0', """gpu id.""")

In [4]:
import rospy
import std_msgs.msg
import sensor_msgs.point_cloud2 as pc2
from sensor_msgs.msg import PointCloud2
from visualization_msgs.msg import Marker
from geometry_msgs.msg import Point

def publish_pc(pc,car,person,cyc):
    pub = rospy.Publisher('/points_raw',PointCloud2,queue_size=10000)
    rospy.init_node('velo_publisher')
    header = std_msgs.msg.Header()
    header.stamp = rospy.Time.now()
    header.frame_id = 'velody'
    points = pc2.create_cloud_xyz32(header,pc)
    '''
    pub2 = rospy.Publisher('/point_seg',PointCloud2,queue_size=10000)
    header = std_msgs.msg.Header()
    header.stamp = rospy.Time.now()
    header.frame_id = 'velody'
    points2 = pc2.create_cloud_xyz32(header,obj)
    '''
    pub_car =rospy.Publisher("/points_raw1",Marker,queue_size=100000)
    marker_car =print_car(pub_car,car)
    
    pub_per =rospy.Publisher("/points_raw2",Marker,queue_size=100000)
    marker_per =print_person(pub_per,person)
    
    pub_cyc=rospy.Publisher("/points_raw3",Marker,queue_size=100000)
    marker_cyc =print_cyc(pub_cyc,cyc)
    
    
    
    r = rospy.Rate(0.1)
    while not rospy.is_shutdown():
        pub.publish(points)
        pub_car.publish(marker_car)
        pub_per.publish(marker_per)
        pub_cyc.publish(marker_cyc)
        r.sleep()
        
def print_car(pub,points):
    triplePoints=[]
    for (x,y,z) in points:
        p = Point()
        p.x = x
        p.y = y
        p.z = z
        triplePoints.append(p)
    marker = Marker()
    marker.header.frame_id = 'velody'
    marker.type = marker.POINTS
    marker.action = marker.ADD
    marker.pose.orientation.w = 1
    marker.points = triplePoints

    marker.scale.x = 0.05
    marker.scale.y = 0.05
    marker.scale.z = 0.05
    marker.color.a = 1.0
    marker.color.r = 1.0
    return marker

def print_person(pub,points):
    triplePoints=[]
    for (x,y,z) in points:
        p = Point()
        p.x = x
        p.y = y
        p.z = z
        triplePoints.append(p)
    marker = Marker()
    marker.header.frame_id = 'velody'
    marker.type = marker.POINTS
    marker.action = marker.ADD
    marker.pose.orientation.w = 1
    marker.points = triplePoints

    marker.scale.x = 0.05
    marker.scale.y = 0.05
    marker.scale.z = 0.05
    marker.color.a = 1.0
    marker.color.g = 1.0
    return marker

def print_cyc(pub,points):
    triplePoints=[]
    for (x,y,z) in points:
        p = Point()
        p.x = x
        p.y = y
        p.z = z
        triplePoints.append(p)
    marker = Marker()
    marker.header.frame_id = 'velody'
    marker.type = marker.POINTS
    marker.action = marker.ADD
    marker.pose.orientation.w = 1
    marker.points = triplePoints

    marker.scale.x = 0.05
    marker.scale.y = 0.05
    marker.scale.z = 0.05
    marker.color.a = 1.0
    marker.color.b = 1.0
    return marker

In [5]:
def visualize_seg(label_map, mc, one_hot=False):
  if one_hot:
    label_map = np.argmax(label_map, axis=-1)
  out = np.zeros(
      (label_map.shape[0], label_map.shape[1], label_map.shape[2], 3))

  for l in range(1, mc.NUM_CLASS):
    out[label_map==l, :] = mc.CLS_COLOR_MAP[l]

  return out

In [6]:
def _normalize(x):
  return (x - x.min())/(x.max() - x.min())

def detect():
  """Detect LiDAR data."""

  os.environ['CUDA_VISIBLE_DEVICES'] = FLAGS.gpu

  with tf.Graph().as_default():
    mc = kitti_squeezeSeg_config()
    mc.LOAD_PRETRAINED_MODEL = False
    mc.BATCH_SIZE = 1 # TODO(bichen): fix this hard-coded batch size.
    model = SqueezeSeg(mc)

    saver = tf.train.Saver(model.model_params)
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
      saver.restore(sess, FLAGS.checkpoint)
      for f in glob.iglob(FLAGS.input_path):
        lidar = np.load(f).astype(np.float32, copy=False)[:, :, :5]
        lidar_mask = np.reshape(
            (lidar[:, :, 4] > 0),
            [mc.ZENITH_LEVEL, mc.AZIMUTH_LEVEL, 1]
        )
        lidar_nor = (lidar - mc.INPUT_MEAN)/mc.INPUT_STD

        pred_cls = sess.run(
            model.pred_cls,
            feed_dict={
                model.lidar_input:[lidar_nor],
                model.keep_prob: 1.0,
                model.lidar_mask:[lidar_mask]
            }
        )
        # save the data
        #print(pred_cls[0].shape)
        #plt.imshow(pred_cls[0])
        
        lidar_xyz = lidar[:,:,:3]
        #print(lidar_xyz.shape)
        lidar_raw = lidar_xyz.reshape(-1,3)
        print (lidar_raw.shape)
        
        Mask = pred_cls[0].reshape(-1,1)
        print(Mask[:,0].shape)
        start =time.time()
        id_0 = np.argwhere(Mask[:,0]==1)
        id_1 = np.argwhere(Mask[:,0]==2)
        id_2 = np.argwhere(Mask[:,0]==3)
        
        car= []
        for index in id_0:
            car.append(lidar_raw[index,:])
        car = np.array(car)
        car = car.reshape(-1,3)
        
        pedestrian=[]
        for index in id_1:
            pedestrian.append(lidar_raw[index,:])
        pedestrian = np.array(pedestrian)
        pedestrian = pedestrian.reshape(-1,3)
        
        cyc=[]
        for index in id_2:
            cyc.append(lidar_raw[index,:])
        cyc = np.array(cyc)
        cyc = cyc.reshape(-1,3)
        
        
        publish_pc(lidar_raw,car,pedestrian,cyc)
        
        
        print(time.time()-start)
        '''
        file_name = f.strip('.npy').split('/')[-1]
        np.save(
            os.path.join(FLAGS.out_dir, 'pred_'+file_name+'.npy'),
            pred_cls[0]
        )

        # save the plot
        depth_map = Image.fromarray(
            (255 * _normalize(lidar[:, :, 3])).astype(np.uint8))
        label_map = Image.fromarray(
            (255 * visualize_seg(pred_cls, mc)[0]).astype(np.uint8))

        blend_map = Image.blend(
            depth_map.convert('RGBA'),
            label_map.convert('RGBA'),
            alpha=0.4
        )

        blend_map.save(
            os.path.join(FLAGS.out_dir, 'plot_'+file_name+'.png'))
        '''

In [7]:
def main(argv=None):
  if not tf.gfile.Exists(FLAGS.out_dir):
    tf.gfile.MakeDirs(FLAGS.out_dir)
  detect()
  print('Detection output written to {}'.format(FLAGS.out_dir))

In [ ]:
if __name__ == '__main__':
    tf.app.run()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
dim is deprecated, use axis instead
------- []
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from /usr/app/code/wk_sp_3/LKN_seg/data/best/model.ckpt-49999
(32768, 3)
(32768,)
